In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cp -r /content/drive/MyDrive/bdb_dataset /content

In [28]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.9 MB/s eta 0:00:00


In [3]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


## The Dataset

In [4]:
from datasets import load_dataset

dataset = load_dataset("audiofolder", data_dir="/content/bdb_dataset")

Resolving data files:   0%|          | 0/674 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
from huggingface_hub import notebook_login

notebook_login()

In [8]:
dataset.push_to_hub("snicolau/bdb_dataset")

Uploading the dataset shards:   0%|          | 0/8 [00:00<?, ?it/s]

Map:   0%|          | 0/85 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Map:   0%|          | 0/85 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

...

Map:   0%|          | 0/84 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Map:   0%|          | 0/84 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/snicolau/bdb_dataset/commit/e2e8c2a0793bc6eeaebf0b27d91d43330654ac5a', commit_message='Upload dataset', commit_description='', oid='e2e8c2a0793bc6eeaebf0b27d91d43330654ac5a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/snicolau/bdb_dataset', endpoint='https://huggingface.co', repo_type='dataset', repo_id='snicolau/bdb_dataset'), pr_revision=None, pr_num=None)

In [10]:
bdb_dataset = dataset["train"].train_test_split(seed=42, shuffle=True, test_size=0.1)
bdb_dataset

DatasetDict({
    train: Dataset({
        features: ['audio', 'label'],
        num_rows: 606
    })
    test: Dataset({
        features: ['audio', 'label'],
        num_rows: 68
    })
})

In [11]:
bdb_dataset["train"][0]

{'audio': {'path': '/content/bdb_dataset/fandango_mall/8_7_fandango_mall.wav',
  'array': array([ 0.19262695,  0.18296814,  0.15827942, ..., -0.12129211,
         -0.04772949, -0.03204346]),
  'sampling_rate': 44100},
 'label': 1}

## Preprocessing the data

In [13]:
from transformers import AutoFeatureExtractor

model_id = "ntu-spml/distilhubert"
feature_extractor = AutoFeatureExtractor.from_pretrained(
    model_id, do_normalize=True, return_attention_mask=True
)

preprocessor_config.json:   0%|          | 0.00/214 [00:00<?, ?B/s]

In [14]:
sampling_rate = feature_extractor.sampling_rate
sampling_rate

16000

In [15]:
from datasets import Audio

bdb_dataset = bdb_dataset.cast_column("audio", Audio(sampling_rate=sampling_rate))

In [17]:
bdb_dataset["train"][0]

{'audio': {'path': '/content/bdb_dataset/fandango_mall/8_7_fandango_mall.wav',
  'array': array([ 0.11958875,  0.20971811,  0.30608091, ..., -0.05702332,
         -0.30277967, -0.1159953 ]),
  'sampling_rate': 16000},
 'label': 1}

In [18]:
import numpy as np

sample = bdb_dataset["train"][0]["audio"]

print(f"Mean: {np.mean(sample['array']):.3}, Variance: {np.var(sample['array']):.3}")

Mean: 0.00415, Variance: 0.0452


In [19]:
inputs = feature_extractor(sample["array"], sampling_rate=sample["sampling_rate"])

print(f"inputs keys: {list(inputs.keys())}")

print(
    f"Mean: {np.mean(inputs['input_values']):.3}, Variance: {np.var(inputs['input_values']):.3}"
)

inputs keys: ['input_values', 'attention_mask']
Mean: 2.74e-09, Variance: 1.0


In [20]:
max_duration = 30.0


def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays,
        sampling_rate=feature_extractor.sampling_rate,
        max_length=int(feature_extractor.sampling_rate * max_duration),
        truncation=True,
        return_attention_mask=True,
    )
    return inputs

In [22]:
bdb_dataset_encoded = bdb_dataset.map(
    preprocess_function,
    remove_columns=["audio"],
    batched=True,
    batch_size=100,
    num_proc=1,
)
bdb_dataset_encoded

Map:   0%|          | 0/606 [00:00<?, ? examples/s]

Map:   0%|          | 0/68 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_values', 'attention_mask'],
        num_rows: 606
    })
    test: Dataset({
        features: ['label', 'input_values', 'attention_mask'],
        num_rows: 68
    })
})

In [23]:
# Replace this with your custom dataset's class names
class_names = ["bolero", "fandango_mall", "fandango_men", "jota", "mateixa"]

# Define id2label and label2id mappings
id2label = {str(i): class_name for i, class_name in enumerate(class_names)}
label2id = {v: k for k, v in id2label.items()}

In [24]:
id2label["3"]

'jota'

## Fine-tuning the model

In [25]:
from transformers import AutoModelForAudioClassification

num_labels = len(id2label)

model = AutoModelForAudioClassification.from_pretrained(
    model_id,
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,
)

config.json:   0%|          | 0.00/1.30k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/94.0M [00:00<?, ?B/s]

Some weights of HubertForSequenceClassification were not initialized from the model checkpoint at ntu-spml/distilhubert and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
from transformers import TrainingArguments

model_name = model_id.split("/")[-1]
batch_size = 8
gradient_accumulation_steps = 1
num_train_epochs = 10

training_args = TrainingArguments(
    f"{model_name}-finetuned-gtzan",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    warmup_ratio=0.1,
    logging_steps=5,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=True,
    push_to_hub=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [29]:
import evaluate
import numpy as np

metric = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [30]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=bdb_dataset_encoded["train"],
    eval_dataset=bdb_dataset_encoded["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)

trainer.train()

<ipython-input-30-02776862b7ac>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy
1,1.506200,1.300939,0.514706
2,1.283800,1.128497,0.588235
3,0.800200,1.052878,0.602941
4,0.638600,0.904661,0.632353
5,0.616500,0.904597,0.661765
6,0.337300,0.655395,0.735294
7,0.290500,0.799556,0.720588
8,0.172200,0.626203,0.808824
9,0.130200,0.538831,0.764706
10,0.061400,0.533763,0.794118


TrainOutput(global_step=760, training_loss=0.6222480579426414, metrics={'train_runtime': 4281.692, 'train_samples_per_second': 1.415, 'train_steps_per_second': 0.177, 'total_flos': 4.134588739776e+17, 'train_loss': 0.6222480579426414, 'epoch': 10.0})